In [1]:
import pandas as pd 
import numpy as np

In [ ]:
raw_sessiondata = pd.read_csv(" .csv", skiprows=6, delimiter = ';')


In [2]:
#Add a column for 'timestamps'
import time
import datetime
import ciso8601

timestamp = []
for i in range(len(raw_sessiondata.index)):
    t = raw_sessiondata['PC-TIME'].iloc[i]
    ts= ciso8601.parse_datetime(t)
    if i == 0:
        t0= time.mktime(ts.timetuple())+ts.microsecond/1000000
        timestamp.append(np.around(t0-t0,3))
    else:
        tnow = time.mktime(ts.timetuple())+ts.microsecond/1000000
        timestamp.append(np.around(tnow-t0,3))

raw_sessiondata['timestamps'] = timestamp

NameError: name 'raw_sessiondata' is not defined

In [ ]:
df_group_msg = raw_sessiondata.groupby('MSG')
df_group_info = raw_sessiondata.groupby('+INFO')
df_group_type = raw_sessiondata.groupby('TYPE')

# Lick Timestamp

In [ ]:
# Get Lick timestamps
df_port1out = df_group_info.get_group('Port1Out')
df_port1out = df_port1out.timestamps.to_numpy()

df_port2out = df_group_info.get_group('Port2Out')
df_port2out = df_port2out.timestamps.to_numpy()

df_port3out = df_group_info.get_group('Port3Out')
df_port3out = df_port3out.timestamps.to_numpy()

df_port4out = df_group_info.get_group('Port4Out')
df_port4out = df_port4out.timestamps.to_numpy()

In [ ]:
#Aggregate all licking according to time
allport = np.hstack((df_port1out,df_port2out, df_port3out,df_port4out))
sorted_allport = np.sort(allport)

# Audio Timstamp

In [ ]:
# Find Audio timestamps ('BNC1High'), process so no duplicates
df_bnc1high = df_group_info.get_group('BNC1High')

In [ ]:
a = df_bnc1high['timestamps'].diff().fillna(df_bnc1high['timestamps']) > 10
timestamps_bnc1high = df_bnc1high.loc[a].timestamps.to_numpy()

# Plot Raster

In [ ]:
count_licks,d=np.histogram(timestamps_lickon, bins=2800000,range=[0, 2800])